<a href="https://colab.research.google.com/github/qahathaway/Automated_Segmentation_PTM-IMAT-SAT/blob/main/Pectoralis_Semantic_Segmentation_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/qahathaway/Automated_Segmentation_PTM-IMAT-SAT.git

Cloning into 'Automated_Segmentation_PTM-IMAT-SAT'...
remote: Enumerating objects: 1982, done.
remote: Counting objects: 100% (4/4), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 1982 (delta 0), reused 2 (delta 0), pack-reused 1978
Receiving objects: 100% (1982/1982), 190.28 MiB | 32.67 MiB/s, done.
Resolving deltas: 100% (58/58), done.
Updating files: 100% (1843/1843), done.
Filtering content: 100% (2/2), 416.60 MiB | 51.93 MiB/s, done.


In [ ]:
!mkdir /content/Automated_Segmentation_PTM-IMAT-SAT/Selected
!mkdir /content/Automated_Segmentation_PTM-IMAT-SAT/Output
!mkdir /content/Automated_Segmentation_PTM-IMAT-SAT/Mask_Infer

# Step 3: Select the frame directly above the aortic arch

In [ ]:
### Load Essential Packages ###
import numpy as np
import pandas as pd
import os
import cv2
from imutils import paths
import argparse
import glob
from glob import iglob

n_white_pix = []
names = []
file_count = []
index = 0

### Select Frame Directly Above the Aorta ###
for filename in sorted(glob.glob('/content/Automated_Segmentation_PTM-IMAT-SAT/Example/**/axial_*.png'), key=lambda name: (name[53:61], int(os.path.basename(name)[14:-4]))):
    image = cv2.imread(filename)
    names.append(filename)
    pathlen, filenamed = os.path.split(filename)
    file_count = len(glob.glob1(pathlen,"axial_*.png"))
    image[image <= 80] = 0
    image[image >= 140] = 0
    cropped = image[165:360, 180:332]
    n_white_pix = np.append(n_white_pix, np.sum(cropped > 0))
    index += 1

### Select the Array of Images to Search and Apply Pixel Intensities to Stratify ###
    if index == file_count:
        lower = int(file_count*0.6)
        upper = int(file_count*0.85)
        minimum = (np.argmin(n_white_pix[lower:upper]) + lower)
        midline = cv2.imread(names[minimum])
        path, filenames = os.path.split(names[minimum])
        path2 = os.path.basename(path)
        cv2.imwrite('/content/Automated_Segmentation_PTM-IMAT-SAT/Selected/' + path2 + "_" + filenames, midline)

        n_white_pix = []
        names = []
        file_count = []
        index = 0

    else:
        continue


# Step 4: Custom semantic segmentation using Pixellib and Mask R-CNN for all remaining participants

In [ ]:
### Load Essential Packages ###
import pixellib
from pixellib.custom_train import instance_custom_training

### Train the MASK R-CNN Model from the Groundtruth Images ###
train_maskrcnn = instance_custom_training()
train_maskrcnn.modelConfig(network_backbone = "resnet50", num_classes= 3, batch_size = 12)
train_maskrcnn.load_pretrained_model("path/to//mask_rcnn_coco.h5")
train_maskrcnn.load_dataset("path/to/manual/segementation")
train_maskrcnn.train_model(num_epochs = 4, augmentation=False,  path_trained_models = "path/to/saved/models/")

'axial_IM-2073_560.png'